### Image rotation in Imviz

In [ ]:
import numpy as np
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.utils.data import download_file

from astroquery.mast import Observations

from jdaviz import Imviz

In [ ]:
cwd = './.'

uris = [
    'mast:HST/product/j9dk11010_drz.fits',
    'mast:HST/product/j9dk11020_drz.fits',
    'mast:JWST/product/jw02731-o001_t017_nircam_clear-f200w_i2d.fits',
    'mast:jwst/product/jw02731-o001_t017_nircam_clear-f187n_i2d.fits'
]

for uri in uris:
    path = cwd + uri.split('/')[-1]
    Observations.download_file(uri, local_path=path)

tess_path = download_file(
    'https://mast.stsci.edu/api/v0.1/Download/file/' + 
    '?uri=mast:TESS/product/tess2023107122157-s0064-3-2-0257-s_ffic.fits', 
    cache=True
)

In [ ]:
imviz = Imviz()

paths = [
    [cwd + 'jw02731-o001_t017_nircam_clear-f200w_i2d.fits', 'JWST/NIRCam F200W'],
    [cwd + 'jw02731-o001_t017_nircam_clear-f187n_i2d.fits', 'JWST/NIRCam F187W'],
    [cwd + 'j9dk11010_drz.fits', 'HST/ACS F658N (1)'],
    [cwd + 'j9dk11020_drz.fits', 'HST/ACS F658N (2)'],
    [tess_path, 'TESS FFI']
]
load_order = [0, 1, 2, 3, 4]

with imviz.batch_load():
    for path, data_label in np.array(paths)[load_order]:
        imviz.load_data(path, data_label=data_label)

po = imviz.plugins['Plot Options']
po.image_color_mode = 'Monochromatic'

colors = np.array(['r', '#40eb34', '#03dbfc', '#03dbfc', 'y'])[load_order]
vmins = np.array([1.2, 5, 0, 0, 0])[load_order]
vmaxes = np.array([8, 150, 3, 3, 3e4])[load_order]
alphas = np.array([1, 1, 1, 1, 0.7])[load_order]

zip_items = zip(
    imviz.app.data_collection, colors, alphas, vmins, vmaxes
)

for i, (data, color, alpha, vmin, vmax) in enumerate(zip_items):
    if not data.label.startswith("CCW"):
        po.layer = data.label
        po.stretch_function = 'arcsinh'
        po.image_opacity = alpha
        po.image_contrast = 1.15
        po.image_color = color
        po.stretch_vmin = vmin
        po.stretch_vmax = vmax

imviz.link_data(link_type='wcs')
imviz.show('sidecar:split-right', height=1200)

near_nebula = SkyCoord(159.22, -58.63, unit=u.deg)
imviz.default_viewer.center_on(near_nebula)

### Generate a set of WCS-only layers

Load the presets for WCS-only layers:

In [ ]:
lc = imviz.plugins['Links Control']

# make option for: North-up, East left
lc._obj.create_north_up_east_left()

# make option for: North-up, East right
lc._obj.create_north_up_east_right()

lc.open_in_tray()

### Making WCS-only layers visible for debugging

WCS-only reference data layers meant for linking/rotation will be invisible in the viewer, unless the user goes through the API to make them visible. Here's how you could do that, if you wanted to check where the rotated WCS lands in the FOV:

In [ ]:
make_wcs_only_layers_visible = False

for i, layer in enumerate(imviz.default_viewer.layers):
    if hasattr(layer.layer, 'meta') and layer.layer.meta.get('_WCS_ONLY', False):
        layer.visible = make_wcs_only_layers_visible
        layer.update()

### Multiple viewers with their own orientations

New viewers can be added and given separate rotation settings: 

In [ ]:
# create new viewer:
imviz.create_image_viewer('imviz-1')

# add a data entry into the new viewer:
imviz.app.add_data_to_viewer('imviz-1', imviz.app.data_collection[1].label)